## 7.5.1 LSTM 구조

**LSTM 순전파**<br>
기울기 소멸 문제를 해결하기 위해 망각 게이트, 입력 게이트 출력 게이트라는 새로운 요소를 은닉층의 각 뉴런에 추가

**망각 게이트** <br>
과거 정보를 어느정도 기억할지 결정. 과거 정보와 현재 데이터를 입력받아 시그모이드를 취한 후 그 값을 과거 정보에 곱해줌. 출력이 0이면 정보는 버리고 1이면 보존

- 계산한 값이 1이면 바로 직전의 정보를 메모리에 유지
- 계산한 값이 0이면 초기화

**입력 게이트** <br>
입력 게이트는 현재 정보를 기억하기 위해 만들어짐. 과거 정보와 현재 데이터를 입력받아 시그모이드와 하이퍼볼릭 탄젠트 함수를 기반으로 현재 정보에 대한 보존량 결정
즉 현재 메모리에 새로운 정보를 반영할지 결정하는 역할을 함

- 계산한 값이 1이면 입력x_t가 들어올 수 있도록 허용
- 계산한 값이 0이면 차단

**셀** <br>
각 단계에 대한 은닉 노드를 메모리셀이라 함. 총압을 사용하여 셀 값을 반영하여 이것으로 기울기 소멸 문제가 해결.<br>
셀 업데이트 방법<br>
:: 망각 게이트와 입력 게이트의 이전 단계 셀 정보를 계산하여 현재 단계의 셀 상태 업데이트

**출력 게이트** <br>
과거 정보와 현재 데이터를 사용하여 출력 결정. 이전 은닉 상태와 t번째 출력을 고려해서 다음 은닉 상태 계산. LSTM에서는 은닉 상태가 그 시점에서의 출력이 됨.<br>
출력게이트는 갱신된 메모리의 출력 값을 제어하는 역할을 함<br>
- 계산한 값이 1이면 의미 있는 결과로 최종 출력
- 계산한 값이 0이면 해당 연산출력을 하지않음



In [1]:
#29. 라이브러리 호출
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter #파라미터 목록을 갖고 있는 라이브러리(패키지)
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math #수학과 관련되어 다양한 함수들과 상수들이 정의되어 있는 라이브러리

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False #GPU 사용에 필요

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor #GPU사용에 필요

torch.manual_seed(125)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(125)

In [2]:
#30. 데이터 전처리
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0,)) #평균을 0.5, 표준편차를 1.0으로 데이터 정규화
])

In [3]:
#31. 데이터 내려받기
from torchvision.datasets import MNIST
#data_dir = '/content/drive/MyDrive/Colab Notebooks/MNIST_DATASET'

download_root='/content/drive/MyDrive/Colab Notebooks/MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True) #내려받을 위치 지정, 전처리 적용, 훈련용데이터, 파일이 없을때 내려받기
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [4]:
#32 데이터셋을 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
vaild_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
#33 변수값 지정
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [6]:
#34 LSTM 셀 네트워크 구축
class LSTMCell(nn.Module):
  def __init__(self, input_size, hidden_size, bias=True):
    super(LSTMCell, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.x2h = nn.Linear(input_size, 4*hidden_size, bias = bias)
    self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias = bias)
    self.reset_parameters()

  def reset_parameters(self):
    std = 1.0/math.sqrt(self.hidden_size)
    for w in self.parameters():
      w.data.uniform_(-std, std)

  def forward(self, x, hidden):
    hx, cx = hidden
    x = x.view(-1, x.size(1))

    gates = self.x2h(x) + self.h2h(hx)
    gates = gates.squeeze()
    ingate, forgetgate, cellgate, outgate = gates.chunk(4,1)

    ingate = F.sigmoid(ingate) #입력 게이트에 시그모이드 활성화 함수 적용
    forgetgate = F.sigmoid(forgetgate ) #망각 게이트에 시그모이드 활성화 함수 적용
    cellgate = F.tanh(cellgate) #셀 게이트에 탄젠트 활성화 함수 적용
    outgate = F.sigmoid(outgate) #출력 게이트에 시그모이드 활성화 함수 적용

    cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate) #4.
    hy = torch.mul(outgate, F.tanh(cy)) #4`
    return(hy,cy)

In [7]:
#35 LSTM 셀의 전반적인 네트워크
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias = True):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim #은닉층의 뉴런/유닛 개수

    self.layer_dim = layer_dim
    self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self,x):
    if torch.cuda.is_available(): #GPU 사용 유무 확인
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) #은닉상태를 0으로 초기화
    else:
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    if torch.cuda.is_available(): #GPU 사용 유무 확인
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) #셀 상태를 0으로 초기화
    else:
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    outs = []
    cn = c0[0,:,:] #셀 상태에 대한 텐서
    hn = h0[0,:,:] #은닉 상태에 대한 텐서

    for seq in range(x.size(1)): #LSTM 셀 계층을 반복하여 쌓아올린다
      hn, cn = self.lstm(x[:,seq,:],(hn,cn))
      outs.append(hn)

    out = outs[-1].squeeze()
    out = self.fc(out)
    return out




In [16]:
#36. 옵티마이저와 손실 함수 지정
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [17]:
#37. 모델 학습 및 성능확인
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader): #훈련데이터셋을 이용한 모델학습
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
      labels = Variable(labels.cuda())
    else:
      images = Variable(images.view(-1, seq_dim, input_dim))
      labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels) #손실함수를 이용하여 오차계산

    if torch.cuda.is_available():
      loss.cuda()

    loss.backward()
    optimizer.step() #파라미터 업데이트
    loss_list.append(loss.item())
    iter += 1

    if iter % 500 == 0: #정확도 계산
      correct = 0
      total = 0
      for images, labels in vaild_loader: #검증 데이터셋을 이용한 모델 성능 검증

        if torch.cuda.is_available():
          images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
          images = Variable(images.view(-1, seq_dim, input_dim))
        outputs = model(images)
        _,predicted = torch.max(outputs.data ,1) #모델을 통과한 결과의 최댓값으로부터 예측결과 가져오기

        total += labels.size(0) #총 레이블 수
        if torch.cuda.is_available():
          correct += (predicted.cpu() == labels.cpu()).sum()
        else:
          correct += (predicted == labels).sum()

      accuracy = 100*correct/total
      print("Iteration: {}. Loss: {}. Accuracy: {}".format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.2252509593963623. Accuracy: 17.09000015258789
Iteration: 1000. Loss: 0.8516435623168945. Accuracy: 74.45999908447266
Iteration: 1500. Loss: 0.29393482208251953. Accuracy: 89.37999725341797
Iteration: 2000. Loss: 0.14795789122581482. Accuracy: 94.13999938964844
Iteration: 2500. Loss: 0.2517714202404022. Accuracy: 94.73999786376953
Iteration: 3000. Loss: 0.21248352527618408. Accuracy: 95.08999633789062
Iteration: 3500. Loss: 0.252878874540329. Accuracy: 95.58999633789062
Iteration: 4000. Loss: 0.057503774762153625. Accuracy: 96.62999725341797
Iteration: 4500. Loss: 0.05089123547077179. Accuracy: 97.19999694824219
Iteration: 5000. Loss: 0.02951515093445778. Accuracy: 97.1500015258789
Iteration: 5500. Loss: 0.133195161819458. Accuracy: 97.73999786376953
Iteration: 6000. Loss: 0.0890934020280838. Accuracy: 97.75
Iteration: 6500. Loss: 0.03826271742582321. Accuracy: 97.77999877929688
Iteration: 7000. Loss: 0.012725875712931156. Accuracy: 97.87999725341797
Iteration: 7

In [21]:
def evaluate(model, val_iter):
  corrects, total, total_loss = 0, 0, 0
  model.eval()
  for images, labels in val_iter:
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
      labels = labels.cuda()  # Move labels to the GPU
    else:
      images = Variable(images.view(-1, seq_dim, input_dim)).to(device)

    logit = model(images).to(device)
    loss = F.cross_entropy(logit, labels, reduction="sum")
    _, predicted = torch.max(logit.data, 1)
    total += labels.size(0)
    total_loss += loss.item()
    corrects += (predicted == labels).sum()

  avg_loss = total_loss / len(val_iter.dataset)
  avg_accuracy = corrects / total
  return avg_loss, avg_accuracy

In [22]:
#39 모델 예측 성능 확인
test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss : %5.2f | Test Accuracy : %5.2f" % (test_loss, test_acc))


Test Loss :  0.06 | Test Accuracy :  0.98
